In [1]:
pip install opencv-contrib-python numpy pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

folders = ["dataset"]
for f in folders:
    os.makedirs(f, exist_ok=True)

# Empty files create
open("students.csv", "w").write("id,name\n")


8

In [3]:
import cv2
import os
import pandas as pd

DATASET_DIR = "dataset"
STUDENTS_CSV = "students.csv"
CASCADE_PATH = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"

def ensure_student_in_csv(student_id, student_name):
    df = pd.read_csv(STUDENTS_CSV)

    if int(student_id) in df["id"].values:
        df.loc[df["id"] == int(student_id), "name"] = student_name
    else:
        df.loc[len(df.index)] = [int(student_id), student_name]

    df.to_csv(STUDENTS_CSV, index=False)

def create_dataset(student_id, student_name, total_images=40):
    ensure_student_in_csv(student_id, student_name)

    face_cascade = cv2.CascadeClassifier(CASCADE_PATH)
    cap = cv2.VideoCapture(0)

    count = 0

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            count += 1
            face_img = gray[y:y+h, x:x+w]
            img_path = f"{DATASET_DIR}/User.{student_id}.{count}.jpg"
            cv2.imwrite(img_path, face_img)

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255,255,255), 2)
            cv2.putText(frame, f"{count}/{total_images}", (10,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        cv2.imshow("Creating Dataset - Press Q to Exit", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if count >= total_images:
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Dataset created successfully!")



In [4]:
create_dataset(1, "Istiak Mamun")


Dataset created successfully!


In [5]:
import cv2
import os
import numpy as np

DATASET_DIR = "dataset"
TRAINER_FILE = "trainer.yml"

def get_images_and_labels(path):
    faces = []
    ids = []

    for filename in os.listdir(path):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            face_id = int(filename.split(".")[1])
            faces.append(img)
            ids.append(face_id)
    return faces, ids

faces, ids = get_images_and_labels(DATASET_DIR)

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(ids))
recognizer.write(TRAINER_FILE)

print("Model trained & Saved!")


Model trained & Saved!


In [6]:
import cv2
import pandas as pd
import os
from datetime import datetime

CASCADE_PATH = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
TRAINER_FILE = "trainer.yml"
STUDENTS_CSV = "students.csv"
ATTENDANCE_CSV = "attendance.csv"

def mark_attendance(student_id, name):
    if os.path.exists(ATTENDANCE_CSV):
        df = pd.read_csv(ATTENDANCE_CSV)
    else:
        df = pd.DataFrame(columns=["id","name","date","time"])

    today = datetime.now().strftime("%Y-%m-%d")
    if ((df["id"] == student_id) & (df["date"] == today)).any():
        return

    now = datetime.now()
    df.loc[len(df.index)] = [student_id, name, now.strftime("%Y-%m-%d"), now.strftime("%H:%M:%S")]
    df.to_csv(ATTENDANCE_CSV, index=False)
    print(f"Attendance Marked for {name}")

def start_attendance():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read(TRAINER_FILE)

    student_df = pd.read_csv(STUDENTS_CSV)
    mapping = dict(zip(student_df.id, student_df.name))

    face_cascade = cv2.CascadeClassifier(CASCADE_PATH)
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x,y,w,h) in faces:
            face_img = gray[y:y+h, x:x+w]
            id_pred, conf = recognizer.predict(face_img)

            if conf < 100:
                name = mapping.get(id_pred, "Unknown")
                mark_attendance(id_pred, name)
                label = f"{name} ({conf:.1f})"
            else:
                label = "Unknown"

            cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,255), 2)
            cv2.putText(frame, label, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        cv2.imshow("Attendance System - Press Q to Exit", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

start_attendance()


In [7]:
create_dataset(2, "Rahim")


Dataset created successfully!


In [8]:
create_dataset(3, "Karim")


Dataset created successfully!
